# Tarea M10 MPAD de Germán Fros

# Problema machine learning: **Predicción de puntos a final de temporada**


### Template para Desarrollo de Modelos siguiendo CRISP-DM

In [41]:
### Configuración Inicial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Importar más librerías según necesidad

pd.set_option('display.max_columns', None)

# 1. Comprensión del Proyecto

## Objetivo del Proyecto
Predecir la cantidad de puntos que un equipo acumulará al final de la temporada

## Contexto del Problema
La directiva de un club uruguayo, junto con su cuerpo técnico, desea diseñar una herramienta que le permita evaluar jornada a jornada el desempeño de su equipo y a qué posiciones en la tabla aspira a final de temporada basado en su rendimiento.

## Criterios de éxito
- MAE <= 6 puntos (≈ 2 partidos de diferencia);
- R² >= 0.6 (explica al menos el 60% de la variación en puntos)

## Supuestos y Restricciones
- Dudas sobre si es suficiente la cantidad de datos de partidos disponible (tres temporadas) para elaborar un modelo efectivo;
- Ver como se comporta con equipos los cuales tenemos menos datos, debido a su ausencia en alguna de las temporadas

# 2. Comprensión de los Datos

## 2.1 Carga de Datos
'''
TODO: 
- Cargar datasets
- Describir fuentes de datos
- Analizar estructura inicial
'''

In [2]:
def cargar_datos(path):
    df = pd.read_excel(path)
    return df

## 2.2 Análisis Exploratorio


In [14]:
df = cargar_datos('../data/raw/sb_matches_2022.xlsx')

In [ ]:
def analisis_exploratorio(df):
    
    print("Dimensiones del DataFrame:", df.shape)

    print("\nResumen:")
    print(df.info(verbose=True))

    print("\nEstadísticas descriptivas:")
    print(df.describe().T)

    print("\nValores Faltantes:")
    print(df.isnull().sum())

    print("\nAnálisis completado.")
    
    return None

In [ ]:
# Análisis completo de todos los archivos Excel
archivos_excel = [
    ('sb_matches_2022.xlsx', 'Partidos | 2022'),
    ('sb_matches_2023.xlsx', 'Partidos | 2023'),
    ('sb_matches_2024.xlsx', 'Partidos | 2024'),
    ('sb_team_match_stats_2022.xlsx', 'Estadísticas de Equipos por Partido | 2022'),
    ('sb_team_match_stats_2023.xlsx', 'Estadísticas de Equipos por Partido | 2023'),
    ('sb_team_match_stats_2024.xlsx', 'Estadísticas de Equipos por Partido | 2024'),
    ('sb_team_season_stats_2022.xlsx', 'Estadísticas de Equipos por Temporada | 2022'),
    ('sb_team_season_stats_2023.xlsx', 'Estadísticas de Equipos por Temporada | 2023'),
    ('sb_team_season_stats_2024.xlsx', 'Estadísticas de Equipos por Temporada | 2024')
]

# Diccionario para almacenar todos los datasets
datasets = {}

for archivo, descripcion in archivos_excel:
    print(f"\n{'='*60}")
    print(f"ANÁLISIS DE: {descripcion}")
    print(f"Archivo: {archivo}")
    print(f"{'='*60}")
    
    try:
        df_temp = cargar_datos(f'../data/raw/{archivo}')
        datasets[archivo] = df_temp
        analisis_exploratorio(df_temp)
        
    except Exception as e:
        print(f"Error al cargar {archivo}: {e}")
        continue


ANÁLISIS DE: Partidos 2022
Archivo: sb_matches_2022.xlsx
Dimensiones del DataFrame: (298, 27)

Resumen:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   match_id               298 non-null    int64  
 1   match_date             298 non-null    object 
 2   kick_off               295 non-null    object 
 3   competition            298 non-null    object 
 4   season                 298 non-null    int64  
 5   home_team              298 non-null    object 
 6   away_team              298 non-null    object 
 7   home_score             295 non-null    float64
 8   away_score             295 non-null    float64
 9   attendance             0 non-null      float64
 10  behind_closed_doors    298 non-null    bool   
 11  neutral_ground         298 non-null    bool   
 12  collection_status      298 non-null    object 
 13  play_

In [44]:
# Resumen comparativo de todos los datasets
print("\n" + "="*80)
print("RESUMEN COMPARATIVO DE TODOS LOS DATASETS")
print("="*80)

for archivo, df in datasets.items():
    print(f"\n{archivo}:")
    print(f"  - Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")
    print(f"  - Valores faltantes: {df.isnull().sum().sum()} total")
    print(f"  - Columnas con valores faltantes: {(df.isnull().sum() > 0).sum()}")
    
    # Identificar columnas clave según el tipo de archivo
    if 'matches' in archivo:
        print(f"  - Equipos únicos: {len(set(df['home_team'].unique().tolist() + df['away_team'].unique().tolist()))}")
    elif 'player' in archivo:
        if 'player_name' in df.columns:
            print(f"  - Jugadores únicos: {df['player_name'].nunique()}")
    elif 'team' in archivo:
        if 'team_name' in df.columns:
            print(f"  - Equipos únicos: {df['team_name'].nunique()}")
            
print("\n" + "="*80)


RESUMEN COMPARATIVO DE TODOS LOS DATASETS

sb_matches_2022.xlsx:
  - Dimensiones: 298 filas x 27 columnas
  - Valores faltantes: 631 total
  - Columnas con valores faltantes: 9
  - Equipos únicos: 16

sb_matches_2023.xlsx:
  - Dimensiones: 300 filas x 27 columnas
  - Valores faltantes: 610 total
  - Columnas con valores faltantes: 3
  - Equipos únicos: 16

sb_matches_2024.xlsx:
  - Dimensiones: 297 filas x 27 columnas
  - Valores faltantes: 598 total
  - Columnas con valores faltantes: 6
  - Equipos únicos: 16

sb_team_match_stats_2022.xlsx:
  - Dimensiones: 588 filas x 91 columnas
  - Valores faltantes: 1 total
  - Columnas con valores faltantes: 1
  - Equipos únicos: 16

sb_team_match_stats_2023.xlsx:
  - Dimensiones: 600 filas x 91 columnas
  - Valores faltantes: 1 total
  - Columnas con valores faltantes: 1
  - Equipos únicos: 16

sb_team_match_stats_2024.xlsx:
  - Dimensiones: 586 filas x 91 columnas
  - Valores faltantes: 1 total
  - Columnas con valores faltantes: 1
  - Equipos

## 2.3 Visualizaciones Iniciales

In [ ]:
def visualizaciones_iniciales(df):
    """
    Crear visualizaciones exploratorias
    """
    # Implementar visualizaciones relevantes
    pass

# 3. Preparación de Datos

## 3.1 Limpieza de Datos

In [ ]:
def limpiar_datos(df):
    """
    Realizar limpieza de datos
    """
    # Implementar limpieza
    pass

## 3.2 Feature Engineering

In [ ]:
def crear_features(df):
    """
    Crear nuevas características
    """
    # Implementar feature engineering
    pass

## 3.3 Preparación para Modelado

In [ ]:
def preparar_datos_modelado(df):
    """
    Preparación final para modelado
    """
    # Implementar preparación final
    pass

# 4. Modelado

## 4.1 Primer Modelo (Especificar tipo)

In [ ]:
def entrenar_modelo(X_train, y_train):
    """
    Entrenar primer modelo
    """
    # Implementar entrenamiento
    pass

# 5. Evaluación

## 5.1 Evaluación Modelo 1

In [ ]:
def evaluar_modelo(modelo, X_test, y_test):
    """
    Evaluar primer modelo
    """
    # Implementar evaluación
    pass

# 6. Despliegue

## 6.1 Preparación para Despliegue

In [ ]:
def preparar_despliegue():
    """
    Preparar modelos para despliegue
    """
    # Implementar preparación
    pass

## 6.2 Documentación de Uso

'''
TODO: Documentar:
- Cómo usar los modelos
- Requerimientos
- Limitaciones
- Mantenimiento necesario
'''

# 7. Conclusiones y Recomendaciones
'''
TODO: Documentar:
- Resultados principales
- Interpretación deportiva
- Limitaciones encontradas
- Mejoras propuestas
- Aplicaciones prácticas
'''


if __name__ == "__main__":
    # Ejecución principal
    print("Iniciando análisis...")